# Installs

In [ ]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


### Levenshtein

This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.3 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 25.23 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 

## Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"krishrana1","key":"ce91845d13bc142d6d456f7ef0f98c92"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=1b398fc8dc560c063076414ba168f48c92829180e674a34c4f33d78ad0caea51
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [ ]:
!kaggle competitions download -c 11-785-s23-hw3p2

100% 15.9G/15.9G [13:43<00:00, 21.4MB/s]
100% 15.9G/15.9G [13:43<00:00, 20.8MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-s23-hw3p2.zip
!ls

11-785-s23-hw3p2  11-785-s23-hw3p2.zip	ctcdecode  sample_data


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here

In [ ]:
len(PHONEMES)

41

IndentationError: ignored

In [ ]:

class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root = '/content/11-785-s23-hw3p2/', partition = 'train-clean-100', context = 0, offset = 0): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        self.mfcc_dir = root + partition + '/mfcc'
        self.transcript_dir = root + partition + '/transcript'  

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''

        self.mfccs, self.transcripts = [], []

        for i in range(len(self.mfcc_files)):

            mfcc        = np.load(self.mfcc_dir + "/" + self.mfcc_files[i])
            mfcc = (mfcc - np.mean(mfcc, axis = 0)) / np.std(mfcc, axis = 0)
    
            transcript  = np.load(self.transcript_dir + "/" + self.transcript_files[i])

            transcript = transcript[1:-1]

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)        

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        # self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        # self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)



        t = []
        for tr in self.transcripts:
          t.append(np.array([self.PHONEMES.index(i) for i in tr]))
        self.transcripts = t
       

    def __len__(self):
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        transcript = self.transcripts[ind] # TODO
        mfcc = torch.tensor(mfcc)
        transcript = torch.tensor(transcript)
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [batch_item[0] for batch_item in batch] # TODO
        # batch of output phonemes
        batch_transcript = [batch_item[1] for batch_item in batch] # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, padding_value=0.0, batch_first=True) # TODO
        lengths_mfcc = [len(item) for item in batch_mfcc] # TODO 
        # lengths_mfcc = torch.tensor(lengths_mfcc)

        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, padding_value=0.0, batch_first=True) # TODO
        lengths_transcript = [len(item) for item in batch_transcript] # TODO
        # lengths_transcript = torch.tensor(lengths_transcript)

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Train Data

### Test Data

In [ ]:
# Test Dataloader
class AudioDatasetTest(torch.utils.data.Dataset):
    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root = '/content/11-785-s23-hw3p2/', partition = "test-clean"): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        self.mfcc_dir = root + partition + '/mfcc'

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        self.length = len(self.mfcc_files) 


        self.mfccs = []

        for i in range(len(self.mfcc_files)):

            mfcc = np.load(self.mfcc_dir + '/' + self.mfcc_files[i])
            mfcc = (mfcc - np.mean(mfcc, axis = 0)) / np.std(mfcc, axis = 0)
       
            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        # self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        # self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        mfcc = torch.FloatTensor(mfcc)
        return mfcc


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = batch

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, padding_value=0.0, batch_first=True) # TODO
        lengths_mfcc = [len(item) for item in batch_mfcc] # TODO 


        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

       

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/11-785-s23-hw3p2' 

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

0

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(partition = 'train-clean-100') #TODO
val_data = AudioDataset(partition='dev-clean') # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest() #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, shuffle = True, collate_fn = train_data.collate_fn, batch_size = BATCH_SIZE)
val_loader = DataLoader(val_data, shuffle = True, collate_fn = val_data.collate_fn, batch_size = BATCH_SIZE)
test_loader = DataLoader(test_data, collate_fn = test_data.collate_fn, batch_size = BATCH_SIZE)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
test_data = AudioDatasetTest() #TODO
test_loader = DataLoader(test_data, collate_fn = test_data.collate_fn, batch_size = BATCH_SIZE)
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1661, 27]) torch.Size([64, 202]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
op = len(LABELS)

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?
        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = 27, hidden_size = 256, num_layers = 3) 
       
        self.classification = nn.Sequential(
            torch.nn.Linear(256, op)
        )

        
        self.logSoftmax = torch.nn.LogSoftmax(dim = 2)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        packed_padding = pack_padded_sequence(x, lengths = lx, batch_first = True, enforce_sorted=False)
        lstm_op = self.lstm(packed_padding)[0]
        op, op_length = torch.nn.utils.rnn.pad_packed_sequence(lstm_op, batch_first = True)
        op = self.classification(op)
        op = self.logSoftmax(op)
        return op, op_length

## Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
from torch.autograd import Variable
class LockedDropout(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        self.dropout = dropout

    def forward(self, x):
        if not self.training or not self.dropout:
            return x
        x, x_lens = pad_packed_sequence(x, batch_first=True)
        
        m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1 - self.dropout)
        mask = Variable(m, requires_grad=False) / (1 - self.dropout)
        mask = mask.expand_as(x)
        return mask * x, None

In [ ]:

class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer
    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = torch.nn.LSTM(batch_first = True, input_size = input_size * 2, hidden_size = hidden_size, 
                          num_layers = 3, bidirectional=True, dropout=0.2)

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        x_pack_padded, x_lens = pad_packed_sequence(x_packed, batch_first=True)
        #print("PBLSTM: ", x_pack_padded.shape)
        
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        x_downsample, x_lens = self.trunc_reshape(x_pack_padded, x_lens)

        #print("Downsample: ", x_downsample.shape)
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.

        # TODO: Pack Padded Sequence. What output(s) would you get?
        x = pack_padded_sequence(x_downsample, x_lens, batch_first=True, enforce_sorted=False)

        #print("PBLSTM: packed: ", x.data.shape)

        # TODO: Pass the sequence through bLSTM
        output, hidden = self.blstm(x)

        # What do you return?
        return output, hidden

    def trunc_reshape(self, x, x_lens): 
        batch_size, max_len, feat_dim = x.size()
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        #print("Max_len: ", max_len)
        if max_len % 2 == 0:
            ds = max_len // 2
            x = x.view(batch_size, ds, feat_dim * 2)
        else:
            x = x[:, :-1, :]
            ds = (max_len - 1) // 2
            x = x.view(batch_size, (max_len - 1) // 2, feat_dim * 2)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        x_lens = torch.clamp(x_lens, max = ds)
        return x, x_lens

# Encoder

In [ ]:

class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()
        
        self.embedding = torch.nn.Sequential(
            # 27, 64
            torch.nn.Conv1d(in_channels = input_size, out_channels = encoder_hidden_size, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.BatchNorm1d(encoder_hidden_size), 
            torch.nn.ReLU(), 

            torch.nn.Conv1d(in_channels = encoder_hidden_size, out_channels = encoder_hidden_size * 2, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.BatchNorm1d(encoder_hidden_size * 2), 
            torch.nn.ReLU(), 

            torch.nn.Conv1d(in_channels = encoder_hidden_size * 2, out_channels = encoder_hidden_size * 2, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.BatchNorm1d(encoder_hidden_size * 2), 
            torch.nn.ReLU()
        )

        #print("Size:", encoder_hidden_size)
        self.pblstm_0 = pBLSTM(input_size = encoder_hidden_size * 2, hidden_size = encoder_hidden_size * 2)

        self.pBLSTMs = torch.nn.Sequential( 
            # TODO: Fill this up with pBLSTMs - What should the input_size be? 

            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...
            *[
                pBLSTM(
                    input_size = encoder_hidden_size * 2 * (2 ** i),
                    hidden_size = encoder_hidden_size * 2 * (2 ** i)
                )
                for i in range(1, 2)
            ]
        )
         
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer

        x = x.transpose(1, 2)
        #print("27: ", x.shape)  # (B, 27, T)
        x = self.embedding(x)

        #print("64: ", x.shape)
        x = x.transpose(1, 2)
        
        #print("64 transpose: ", x.shape) # (B, 64, T)

        # Pack and pass to sequence model
        # TODO: Pack Padded Sequence
        x = pack_padded_sequence(x, x_lens, batch_first = True, enforce_sorted = False)

        #print(x.data.shape)

        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        x, _ = self.pblstm_0.forward(x)
        for layer in self.pBLSTMs:
            x, _ = layer.forward(x)
        # TODO: Pad Packed Sequence
        x, encoder_lens = pad_packed_sequence(x, batch_first = True)
        encoder_outputs = x

        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

# Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), 
            torch.nn.BatchNorm1d(embed_size), 
            PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            torch.nn.Linear(embed_size, 512), 
            torch.nn.ReLU(), 
            torch.nn.Dropout(0.2),
            PermuteBlock(), 
            torch.nn.BatchNorm1d(512), 
            PermuteBlock(),
            torch.nn.Linear(512, 128), 
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            PermuteBlock(), 
            torch.nn.BatchNorm1d(128), 
            PermuteBlock(),
            torch.nn.Linear(128, output_size)
            #Use Permute Block before and after BatchNorm1d() to match the size
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        output = self.mlp(encoder_out)
        output = self.softmax(output)
        #TODO Think what should be the final output of the decoder for the classification 
        return output

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size = 512, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            tat.TimeMasking(20),
            tat.FrequencyMasking(20)
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        = Encoder(input_size, 64)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size, output_size)# TODO: Initialize Decoder 

        
    
    def forward(self, x, lengths_x):
        
        if self.training:
            x = self.augmentations(x)

        #print("ASR: ", x.shape)
        #print("ASR: ", lengths_x.shape)


        encoder_out, encoder_lens   = self.encoder.forward(x, lengths_x)
        decoder_out                 = self.decoder.forward(encoder_out)

        return decoder_out, encoder_lens

## INIT
(If trying out the basic Network)

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                          Kernel Shape    Output Shape     Params  Mult-Adds
Layer                                                                       
0_lstm                               -    [79746, 256]  1.344512M  1.338368M
1_classification.Linear_0    [256, 41]  [64, 1661, 41]    10.537k    10.496k
2_logSoftmax                         -  [64, 1661, 41]          -          -
----------------------------------------------------------------------------
                         Totals
Total params          1.355049M
Trainable params      1.355049M
Non-trainable params        0.0
Mult-Adds             1.348864M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[79746, 256]",1344512.0,1338368.0
1_classification.Linear_0,"[256, 41]","[64, 1661, 41]",10537.0,10496.0
2_logSoftmax,-,"[64, 1661, 41]",NaN,NaN


## INIT ASR

In [ ]:
model = ASRModel(input_size  = 27, embed_size  = 512, output_size = len(PHONEMES)).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential(
    (0): TimeMasking()
    (1): FrequencyMasking()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): Conv1d(27, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
    (pblstm_0): pBLSTM(
      (blstm): LSTM(256, 128, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(512, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
      )
    )
  )
  (decoder): Dec

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.TimeMasking_0,-,"[64, 1661, 27]",NaN,NaN
1_augmentations.FrequencyMasking_1,-,"[64, 1661, 27]",NaN,NaN
2_encoder.embedding.Conv1d_0,"[27, 64, 3]","[64, 64, 1661]",5248.0,8610624.0
3_encoder.embedding.BatchNorm1d_1,[64],"[64, 64, 1661]",128.0,64.0
4_encoder.embedding.ReLU_2,-,"[64, 64, 1661]",NaN,NaN
5_encoder.embedding.Conv1d_3,"[64, 128, 3]","[64, 128, 1661]",24704.0,40820736.0
6_encoder.embedding.BatchNorm1d_4,[128],"[64, 128, 1661]",256.0,128.0
7_encoder.embedding.ReLU_5,-,"[64, 128, 1661]",NaN,NaN
8_encoder.embedding.Conv1d_6,"[128, 128, 3]","[64, 128, 1661]",49280.0,81641472.0


In [ ]:
ckpt = torch.load('/content/best_model-5.pth')
model.load_state_dict(ckpt['model_state_dict'])

# Training Config

In [ ]:
config = {
    "beam_width" : 2,
    "lr" : 2e-3,
    "epochs" : 100
    } # Feel free to add more items here

In [ ]:
#TODO


criterion = torch.nn.CTCLoss(zero_infinity = True)
# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), config['lr']) # What goes in here?
#optimizer.load_state_dict(ckpt['optimizer_state_dict'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, beam_width=config['beam_width'], log_probs_input=True)#TODO 

scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7, patience=2) #TODO
#scheduler.load_state_dict(ckpt['scheduler_state_dict'])
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    
    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []
    
    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        beam_slice = beam_results[i,0,:out_seq_len[i,0]]
        pred_str = []
        pred_str = [PHONEME_MAP[i] for i in beam_slice]
        pred_strings.append(pred_str)
    
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        label_slice = label[i,0:label_lens[i]] #TODO 
        label_string = [PHONEME_MAP[k] for k in label_slice]
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)
    print(loss)
    h = torch.permute(h, (1, 0, 2))

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 593, 41])
torch.Size([593, 64, 41]) torch.Size([64, 269])
tensor(25.3000, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(25.3000, device='cuda:0', grad_fn=<MeanBackward0>)
153.484375


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="cce3530b8ee00a25da9393f98f6b0142435348a3")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "pblstm-model-nlayers3-conv2-dp-2-ckpt-sub", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #run_id = "zputl6zb", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-krana", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: krana (cmu-idl). Use `wandb login --relogin` to force relogin


# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)


        if not torch.isfinite(loss).all():
            print(f'WARNING: non-finite loss, skipping update for batch {i}')
            print(loss)
            print(lh >= ly)
            continue

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
# test code to check shapes# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)
    h = torch.permute(h, (1, 0, 2))

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 676, 41])
torch.Size([676, 64, 41]) torch.Size([64, 226])
tensor(27.3597, device='cuda:0', grad_fn=<MeanBackward0>)
168.109375


### Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = 'chekpoint.pth'#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = 'best_model.pth'#TODO set best model path 

In [ ]:
torch.cuda.empty_cache()
gc.collect()

print(decoder)
#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    
    # curr_lr = config['lr'] #TODO
    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss  = train_model(model, train_loader, criterion, optimizer)             #TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder) #TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,  
        'valid_dist': valid_dist, 
        'valid_loss': valid_loss, 
        'lr'        : curr_lr
    })
    
    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.run.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.run.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10

test_decoder    = CTCBeamDecoder(LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)#TODO 
test_results = []
model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string = decode_prediction(h, lh, test_decoder)
    #TODO save the output in results array.
    test_results.extend(prediction_string)
    
    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:26<00:00,  1.55it/s]


In [ ]:
test_results_new = ["".join(arr) for arr in test_results]


In [ ]:
data_dir = '/content/11-785-s23-hw3p2/test-clean/random_submission.csv'
df = pd.read_csv(data_dir)
df.label = test_results_new
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c 11-785-s23-hw3p2 -f submission.csv -m "I made it!"


100% 210k/210k [00:02<00:00, 78.7kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)